In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt
import os
import tools

sc.logging.print_header()

os.chdir(os.path.expanduser('/home/jovyan/Zach/scanpy/'))
adata = sc.read_h5ad('h5ad/meninges_integrated.h5ad')
adata

scanpy==1.8.2 anndata==0.7.6 umap==0.5.1 numpy==1.21.6 scipy==1.5.2 pandas==1.3.5 scikit-learn==0.23.2 statsmodels==0.12.1 python-igraph==0.8.3 leidenalg==0.8.2 pynndescent==0.5.2


AnnData object with n_obs × n_vars = 10839 × 3266
    obs: 'sample_id', 'condition', 'site', 'sequencing_batch', 'scrublet_score', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'gmm_pct_count_clusters_keep', 'is_doublet', 'filter_rna', 'S_score', 'G2M_score', 'phase', 'leiden', 'GSE23925_DARK_ZONE_VS_NAIVE_BCELL_DN', 'GSE23925_DARK_ZONE_VS_NAIVE_BCELL_UP', 'GSE23925_LIGHT_ZONE_VS_DARK_ZONE_BCELL_DN', 'GSE23925_LIGHT_ZONE_VS_DARK_ZONE_BCELL_UP', 'GSE23925_LIGHT_ZONE_VS_NAIVE_BCELL_DN', 'GSE23925_LIGHT_ZONE_VS_NAIVE_BCELL_UP', 'leiden_R', 'celltype', 'keep'
    var: 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'celltype_colors', 'condition_colors', 'filter_rna_colors', 'hvg', 'is_doublet_colors', 'keep_colors', 'leiden', 'leiden_R_colors', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'sample_id_colors', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'dist

In [2]:
celltypes = list(adata.obs['celltype'].cat.categories)

In [4]:
from tqdm import tqdm
degs = {}
for c in tqdm(celltypes):
    tmp = adata[adata.obs['celltype'] == c].copy()
    try:
        sc.tl.rank_genes_groups(tmp, groupby = 'condition', method = 'wilcoxon', pts = True)
        degs[c] = tools.returnDEres(tmp, column = 'VSV')
    except:
        pass

100%|██████████| 27/27 [00:19<00:00,  1.40it/s]


In [5]:
degs['B_GC']

,scores,logfoldchanges,pvals,pvals_adj,pts_VSV,pts_rest
Ly6a,10.456399,3.364757,1.369618e-25,4.948885e-22,0.852,0.237288
Dennd4a,10.271609,2.851868,9.461317e-25,2.279126e-21,0.920,0.449153
Lncpint,10.037996,3.608897,1.037602e-23,2.249521e-20,0.768,0.161017
Arhgap15,9.889429,2.576891,4.626580e-23,8.358687e-20,0.888,0.500000
Inpp5d,9.753986,2.402841,1.773635e-22,2.957878e-19,0.904,0.474576
...,...,...,...,...,...,...
H3f3b,-9.426929,-1.125801,4.222727e-21,5.385219e-18,0.976,0.991525
Jund,-10.364004,-2.069431,3.615038e-25,9.796753e-22,0.812,0.974576
H3f3a,-10.378703,-1.286245,3.099558e-25,9.599773e-22,0.984,1.000000
Cd79a,-10.853278,-1.245173,1.923979e-27,8.342371e-24,0.992,0.983051


In [25]:
b_g = degs['B_GC']
b_g[(b_g.pts_VSV >= .25) & (b_g.pvals_adj < .05) & (b_g.logfoldchanges > 1)]

,scores,logfoldchanges,pvals,pvals_adj,pts_VSV,pts_rest
Ly6a,10.456399,3.364757,1.369618e-25,4.948885e-22,0.852,0.237288
Dennd4a,10.271609,2.851868,9.461317e-25,2.279126e-21,0.920,0.449153
Lncpint,10.037996,3.608897,1.037602e-23,2.249521e-20,0.768,0.161017
Arhgap15,9.889429,2.576891,4.626580e-23,8.358687e-20,0.888,0.500000
Inpp5d,9.753986,2.402841,1.773635e-22,2.957878e-19,0.904,0.474576
...,...,...,...,...,...,...
Slc10a7,3.280027,1.879986,1.037970e-03,4.611311e-02,0.304,0.093220
Cstf3,3.278977,1.543391,1.041840e-03,4.619037e-02,0.352,0.144068
Ppp6r3,3.266378,1.304876,1.089327e-03,4.800125e-02,0.388,0.169492
Ptbp2,3.264803,2.030298,1.095402e-03,4.814656e-02,0.300,0.093220


In [24]:
list(b_g[(b_g.pts_VSV >= .25) & (b_g.pvals_adj < .05) & (b_g.logfoldchanges > 1)].index)

['Ly6a',
 'Dennd4a',
 'Lncpint',
 'Arhgap15',
 'Inpp5d',
 'Dock2',
 'Bank1',
 'Stat4',
 'Aff3',
 'Immp2l',
 'Cd86',
 'Plac8',
 'Pde4b',
 'H2-Q7',
 'Stat1',
 'Dock10',
 'Man1a',
 'Maml2',
 'Phlpp1',
 'H2-K1',
 'Mir142hg',
 'Mycbp2',
 'Gimap4',
 'Ifi27l2a',
 'Elmo1',
 'Inpp4b',
 'Prkcb',
 'Uvrag',
 'Pde3b',
 'Cmip',
 'Psmb9',
 'Nfkb1',
 'Serpina3g',
 'Nsd3',
 'Sik3',
 'Grb2',
 'Zbp1',
 'Bach2',
 'Ms4a4c',
 'Ppp1r16b',
 'Epb41',
 'Zbtb20',
 'Akt3',
 'Fryl',
 'Gm31243',
 'Pan3',
 'Atxn1',
 'Cblb',
 'Pik3ap1',
 'Ripor2',
 'Setbp1',
 'Satb1',
 'Celf2',
 'Psmb8',
 'Rapgef6',
 'Ccr6',
 'Itpr1',
 'Fyn',
 'Fbxo11',
 'Etv6',
 'Exoc4',
 'Rab43',
 'Csnk1g3',
 'Fnbp1',
 'Sipa1l1',
 'Ankrd44',
 'Vps37b',
 'Fli1',
 'Hivep2',
 'Myo1e',
 'Stat3',
 'Ssh2',
 'Arih1',
 'Rtn3',
 'Btbd9',
 'Baz2b',
 'Shisa5',
 'Gimap6',
 'Antxr2',
 'Ly6c2',
 'Ifi47',
 'Bmp2k',
 'Rftn1',
 'Cmss1',
 'Gpr132',
 'Smyd3',
 'Nlrc5',
 'Sp140',
 'Fbxl17',
 'Kdm2b',
 'Plekhm3',
 'Cnot4',
 'Lpxn',
 'Aff4',
 'Wnk1',
 'Camk2d',
 'Fam49b

In [26]:
b_p = degs['B_Plasma']
b_p[(b_p.pts_VSV >= .25) & (b_p.pvals_adj < .05) & (b_p.logfoldchanges > 0)]

,scores,logfoldchanges,pvals,pvals_adj,pts_VSV,pts_rest
Cmss1,6.083242,9.871777,1.177762e-09,0.000020,0.988372,0.066667
Gm42418,5.892246,2.434816,3.809822e-09,0.000028,1.000000,1.000000
Camk1d,5.691699,7.339222,1.257813e-08,0.000055,0.930233,0.066667
Nme2,5.605751,7.449582,2.073542e-08,0.000059,0.930233,0.266667
Cdk8,5.424304,4.264377,5.818081e-08,0.000082,0.930233,0.333333
...,...,...,...,...,...,...
Cdc42se2,3.437939,1.669863,5.861595e-04,0.046893,0.813953,0.933333
Srp72,3.428389,1.645240,6.071741e-04,0.048042,0.790698,0.666667
Eif2s2,3.428389,1.219677,6.071741e-04,0.048042,0.895349,1.000000
Eny2,3.428389,1.530326,6.071741e-04,0.048042,0.813953,0.800000


In [27]:
list(b_p[(b_p.pts_VSV >= .25) & (b_p.pvals_adj < .05) & (b_p.logfoldchanges > 0)].index)

['Cmss1',
 'Gm42418',
 'Camk1d',
 'Nme2',
 'Cdk8',
 'Rnasek',
 'Atp5o',
 'Tpt1',
 'Atp6v0c',
 'Ndfip1',
 'Ifi27l2a',
 'Rack1',
 'Ftl1',
 'Eef1b2',
 'Glcci1',
 'Sik3',
 'Sec61b',
 'Fau',
 'Oaz1',
 'Psmb8',
 'Sub1',
 'Arhgap15',
 'Sec11c',
 'Calm1',
 'Tomm6',
 'Slc25a3',
 'Atp5b',
 'Srsf7',
 'Actb',
 'Naca',
 'Atxn1',
 'Nme1',
 'Cox4i1',
 'H2-Q7',
 'Eif4a1',
 'Gphn',
 'Eef1a1',
 'Chchd2',
 'Cox5a',
 'Dynll1',
 'Eef2',
 'Ssr2',
 'H3f3a',
 'Ostc',
 'Calr',
 'Ndufb7',
 'Npm1',
 'Manf',
 'Ankrd11',
 'Dock2',
 'Srsf3',
 'Ppia',
 'Atp5g1',
 'Eif5a',
 'Ube2s',
 'Plac8',
 'Rbm3',
 'Eef1d',
 'Cct3',
 'Stag1',
 'Snd1',
 'Atp5h',
 'Timm23',
 'Ndufb11',
 'Foxn3',
 'Blnk',
 'Gng12',
 'Txn1',
 'Psmb1',
 'Fnbp1',
 'Tmsb4x',
 'Myl6',
 'Abcf1',
 'Clint1',
 'Exoc4',
 'Psma2',
 'Gapdh',
 'Eif3m',
 'Eef1g',
 'Top1',
 'Eif3h',
 'Psmb5',
 'Nbas',
 'Gm11084',
 'Aarsd1',
 'Lman2',
 'Gnas',
 'Slc25a5',
 'Erp29',
 'Calm3',
 'Ell2',
 'Atp5f1',
 'Wdr33',
 'Psma7',
 'Inpp4b',
 'Mcl1',
 'Psmb4',
 'Hsp90ab1',
 'Clta',